In [1]:
import numpy as np
import os

import random
import math

import matplotlib.pyplot as plt
import string
import astropy.units as u
import astropy.constants as const
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

In [2]:
import sys

from s1s2_0311 import *
from read import *
from recoil_1223 import *


In [3]:
nuclei = 'Argon'

In [12]:

pcles = ['Be7_861']
binnum = 400
start = -12
ebind = False
metallicity = 'low'
smear =  False

sigma_percentage = 0.1
if smear and nuclei == 'Xenon':
    corr = '_smear'
elif smear and nuclei == 'Argon':
    corr = '_smear'+str(sigma_percentage)
elif smear==False:
    corr = ''

write = True
for pcle  in pcles:
    recoil_type = 'NR'#get_recoil_type(pcle)
    #recoil_type = 'ER'
    bind = get_Ebind(pcle,recoil_type, False)
    if pcle in ['dsnb', 'nubb','Rn222', 'Kr85']:
        fig2, ax2 = plt.subplots()
        
        Er_keV, oiginal_pdfs = read_pcle_pdf(pcle, nuclei, '', recoil_type = '', 
                                metallicity_model = metallicity, ebinding = True, plot = True, corr = '')
        #oiginal_pdfs = oiginal_pdfs /u.keV/u.tonne/u.yr
        #Er_GeV = (Er_keV*u.keV).to(u.GeV)
        ax2.loglog((Er_keV*u.keV).to(u.GeV), oiginal_pdfs)
        if pcle in ['nubb','Rn222', 'Kr85']:
            if pcle in ['nubb']:
                start = np.log10(min((Er_keV*u.keV).to(u.GeV)).value)#np.log10(((Er_keV[1]*u.keV).to(u.GeV)).value)#
            Er_GeV, oiginal_pdfs = linear_Itp_pdf(start, np.log10(max((Er_keV*u.keV).to(u.GeV)).value), binnum, 
                                              (Er_keV*u.keV).to(u.GeV), oiginal_pdfs/u.keV/u.tonne/u.yr)
            
            ax2.loglog( Er_GeV, oiginal_pdfs )
    
    else:
        Er_GeV =get_Er(pcle, nuclei, recoil_type, start=start, binnum= binnum)
    
        print('max Er = ',max(Er_GeV).to(u.keV), '\n')
        bind = get_Ebind(pcle,recoil_type, ebind)
        Enu_MeV, dN_dEnu_percm2sMeV = flux_norm(pcle, norm_unit = (u.cm**2 * u.s)**(-1), metallicity_model = metallicity)
        #print(dN_dEnu_percm2sMeV)
        oiginal_pdfs = get_dR_dEr_pertonyrkeV(Er_GeV, nuclei, Enu_MeV.to(u.GeV), dN_dEnu_percm2sMeV, recoil_type, pcle,
                                                  ebinding = ebind, corr = '')

        
    if smear == True:

        E_r, pdf = add_smearing(Er_GeV, oiginal_pdfs, nuclei = nuclei, sigma_percentage = sigma_percentage,plot = True)
        pdf_unit = pdf.unit
     
    else:
        E_r = Er_GeV.to(u.keV)
        pdf = oiginal_pdfs
        

    data_out_pdf = np.array([E_r.to(u.keV).value, pdf.value]).T
    
    if get_solar_components(pcle):
        print('solar components, metallicity matters')
        pdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'pdf'),
                                        pcle+'_'+recoil_type+bind+nuclei+'_pdf_'+metallicity+corr+'.txt')
        cdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'cdf'),
                                            pcle+'_'+recoil_type+bind+nuclei+'_cdf_'+metallicity+corr+'.txt')
    else:
        print('not solar components')
        pdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'pdf'),
                                        pcle+'_'+recoil_type+bind+nuclei+'_pdf'+corr+'.txt')
        cdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'cdf'),
                                            pcle+'_'+recoil_type+bind+nuclei+'_cdf'+corr+'.txt')
    

    dEr_GeVs = np.diff(Er_GeV) 
    Er_GeV_centers =(Er_GeV[1:]+Er_GeV[:-1])/2
    
    if smear == True:
        cdf = get_cdf(pdf, dEr_GeVs, 'survival')
    else:
        cdf = get_cdf(pdf[:-1], dEr_GeVs, 'survival')
    
    
    E_centers = Er_GeV_centers.to(u.keV).value
    data_out_cdf = np.array([E_centers, cdf/max(cdf)]).T
    print(pdf_path, cdf_path)
    if write:
        write_file_data(pdf_path, data_out_pdf,10, append= False)
        write_file_data(cdf_path, data_out_cdf,10, append= False)


max Er =  0.09999999999999999 keV 

Be7_861
AGSS09-SFII
Be7_861 mono energy
nucleon mass:  37.211326403360076 GeV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
solar components, metallicity matters
survival
../real_data_nest/pdf/Be7_861_NR_Argon_pdf_low.txt ../real_data_nest/cdf/Be7_861_NR_Argon_cdf_low.txt


# sum over CNO

In [26]:
ER_pcles = ['N13','O15' ,'F17']
pcles = ER_pcles
nuclei = 'Xenon'
pdfs = []
Er_GeV_tot = []
binnum = 400
start = -12
ebind = False
pdfs = []
metallicity = 'high'

Er_GeV_uni =get_Er('O15', nuclei, 'NR', start=start, binnum= binnum)
smear = False
sigma_percentage = 0.1
if smear and nuclei == 'Xenon':
    corr = '_smear'
elif smear and nuclei == 'Argon':
    corr = '_smear'+str(sigma_percentage)
elif smear==False:
    corr = ''
    
    
for pcle in pcles:
    recoil_type ='NR'#get_recoil_type(pcle)
    
    #print(Er_GeV)
    bind = get_Ebind(pcle,recoil_type, ebind)
    Enu_MeV, dN_dEnu_percm2sMeV = flux_norm(pcle, norm_unit = (u.cm**2 * u.s)**(-1), metallicity_model = metallicity)
    print(dN_dEnu_percm2sMeV)
    
    pdf = get_dR_dEr_pertonyrkeV(Er_GeV_uni, nuclei, Enu_MeV.to(u.GeV), dN_dEnu_percm2sMeV, recoil_type, pcle,
                                              ebinding = ebind, corr = corr)
    pdfs.append(pdf)


N13
GS98-SFII
N13
[9.5613920e+04 3.7962000e+05 8.4777360e+05 1.4958952e+06 2.3197816e+06
 3.3152000e+06 4.4784800e+06 5.8048560e+06 7.2907760e+06 8.9320960e+06
 1.0724672e+07 1.2664656e+07 1.4748200e+07 1.6971160e+07 1.9329984e+07
 2.1820528e+07 2.4439240e+07 2.7182272e+07 3.0046960e+07 3.3024720e+07
 3.6117920e+07 3.9320640e+07 4.2629920e+07 4.6039840e+07 4.9550400e+07
 5.3152720e+07 5.6849760e+07 6.0632640e+07 6.4501360e+07 6.8450000e+07
 7.2478560e+07 7.6578160e+07 8.0751760e+07 8.4990480e+07 8.9294320e+07
 9.3660320e+07 9.8085520e+07 1.0256400e+08 1.0709280e+08 1.1167192e+08
 1.1629544e+08 1.2096040e+08 1.2566680e+08 1.3040872e+08 1.3518320e+08
 1.3998728e+08 1.4482096e+08 1.4967832e+08 1.5455640e+08 1.5945224e+08
 1.6436584e+08 1.6929128e+08 1.7422856e+08 1.7917472e+08 1.8412384e+08
 1.8907592e+08 1.9402504e+08 1.9897416e+08 2.0391736e+08 2.0885168e+08
 2.1377712e+08 2.1868776e+08 2.2358360e+08 2.2846168e+08 2.3331904e+08
 2.3815568e+08 2.4296568e+08 2.4774608e+08 2.5249984e+08 2.

F17
GS98-SFII
F17
[2.2433280e+02 8.9446080e+02 2.0060784e+03 3.5549904e+03 5.5371120e+03
 7.9476960e+03 1.0782768e+04 1.4037912e+04 1.7709816e+04 2.1792960e+04
 2.6284032e+04 3.1178616e+04 3.6472848e+04 4.2162312e+04 4.8242592e+04
 5.4710376e+04 6.1559040e+04 6.8790240e+04 7.6396800e+04 8.4367680e+04
 9.2708400e+04 1.0141344e+05 1.1047728e+05 1.1989440e+05 1.2966480e+05
 1.3977744e+05 1.5023232e+05 1.6102944e+05 1.7216328e+05 1.8362280e+05
 1.9541352e+05 2.0752440e+05 2.1995544e+05 2.3270112e+05 2.4575592e+05
 2.5911984e+05 2.7279288e+05 2.8675848e+05 3.0102216e+05 3.1558392e+05
 3.3042720e+05 3.4555752e+05 3.6096936e+05 3.7665720e+05 3.9262104e+05
 4.0884984e+05 4.2534912e+05 4.4211336e+05 4.5913152e+05 4.7640360e+05
 4.9392960e+05 5.1170400e+05 5.2972128e+05 5.4798144e+05 5.6646240e+05
 5.8523040e+05 6.0416400e+05 6.2337360e+05 6.4274880e+05 6.6240000e+05
 6.8221680e+05 7.0230960e+05 7.2256800e+05 7.4299200e+05 7.6369200e+05
 7.8455760e+05 8.0564400e+05 8.2689600e+05 8.4831360e+05 8.

In [27]:
smear = False
oiginal_pdfs = sum(pdfs)
Er_GeV = Er_GeV_uni
if smear == True:
    E_r, pdf = add_smearing(Er_GeV, oiginal_pdfs, nuclei = nuclei, sigma_percentage = 0.07,plot = True)
else:
    E_r = Er_GeV.to(u.keV)
    pdf = oiginal_pdfs

pcle = 'CNO'
if get_solar_components(pcle):
    print('solar components, metallicity matters')
    pdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'pdf'),
                                    pcle+'_'+recoil_type+bind+nuclei+'_pdf_'+metallicity+corr+'.txt')
    cdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'cdf'),
                                        pcle+'_'+recoil_type+bind+nuclei+'_cdf_'+metallicity+corr+'.txt')
else:
    print('not solar components')
    pdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'pdf'),
                                    pcle+'_'+recoil_type+bind+nuclei+'_pdf'+corr+'.txt')
    cdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'cdf'),
                                        pcle+'_'+recoil_type+bind+nuclei+'_cdf'+corr+'.txt')
    
data_out_pdf = np.array([E_r.to(u.keV).value, pdf.value]).T
dEr_GeVs = np.diff(Er_GeV) 
Er_GeV_centers =(Er_GeV[1:]+Er_GeV[:-1])/2

if smear == True:
    cdf = get_cdf(pdf, dEr_GeVs, 'survival')
else:
    cdf = get_cdf(pdf[:-1], dEr_GeVs, 'survival')
    
E_centers = Er_GeV_centers.to(u.keV).value

data_out_cdf = np.array([E_centers, cdf/max(cdf)]).T
print(max(cdf), pdf_path, cdf_path)
write_file_data(pdf_path, data_out_pdf,10, append= False)
write_file_data(cdf_path, data_out_cdf,10, append= False)


solar components, metallicity matters
survival
1398.2676525270765 1 / (t yr) ../real_data_nest/pdf/CNO_NR_Xenon_pdf_high.txt ../real_data_nest/cdf/CNO_NR_Xenon_cdf_high.txt


In [4]:
NR_pcles = ['atmNu_Kamioka_FLUKA', 'atmNu_LNGS_FLUKA']

pcles =NR_pcles

pdfs = []
Er_GeV_tot = []
binnum = 400
start = -9
ebind = True
pdfs = []
metallicity = ''
#Er_GeV =np.logspace(-9, np.log10(2000e-6), 400)*u.GeV
for pcle  in pcles:
    recoil_type =get_recoil_type(pcle)
    Er_GeV =get_Er(pcle, nuclei, recoil_type, start=start, binnum= binnum)
    
    #print(Er_GeV)
    bind = get_Ebind(pcle,recoil_type, ebind)
    Enu_MeV, dN_dEnu_percm2sMeV = flux_norm(pcle, norm_unit = (u.cm**2 * u.s)**(-1),metallicity_model = metallicity)
    #print(dN_dEnu_percm2sMeV)
    
    pdf = get_dR_dEr_pertonyrkeV(Er_GeV, nuclei, Enu_MeV.to(u.GeV), dN_dEnu_percm2sMeV, recoil_type, pcle,
                                              ebinding = ebind)
    #print(Er_GeV)
   
    data_out_pdf = np.array([Er_GeV.to(u.keV).value, pdf.value]).T
    pdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'pdf'),
                                    pcle+'_'+recoil_type+bind+nuclei+'_pdf'+metallicity+'.txt')
    cdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'cdf'),
                                    pcle+'_'+recoil_type+bind+nuclei+'_cdf'+metallicity+'.txt')


    dEr_GeVs = np.diff(Er_GeV) 
    Er_GeV_centers =(Er_GeV[1:]+Er_GeV[:-1])/2
    cdf = get_cdf(pdf[:-1], dEr_GeVs, 'survival')
    E_centers = Er_GeV_centers.to(u.keV).value

    data_out_cdf = np.array([E_centers, cdf/max(cdf)]).T
    print(pdf_path, cdf_path)
    write_file_data(pdf_path, data_out_pdf,10, append= False)
    write_file_data(cdf_path, data_out_cdf,10, append= False)
    

atmNu_Kamioka_FLUKA
atmNu_Kamioka_FLUKA
nucleon mass:  37.211326403360076 GeV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
survival
./real_data_nest/pdf/atmNu_Kamioka_FLUKA_NR_Argon_pdf.txt ./real_data_nest/cdf/atmNu_Kamioka_FLUKA_NR_Argon_cdf.txt
atmNu_LNGS_FLUKA
atmNu_LNGS_FLUKA
nucleon mass:  37.211326403360076 GeV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
survival
./real_data_nest/pdf/atmNu_LNGS_FLUKA_NR_Argon_pdf.txt ./real_data_nest/cdf/atmNu_LNGS_FLUKA_NR_Argon_cdf.txt


# write atm_avg pdf

In [17]:

detectors = ['CJPL','Kamioka', 'LNGS', 'SURF','SNOlab']#
years = ['2009','2014', 'avg']

pdfs = []
Er_GeV_tot = []
binnum = 400
start = -9
ebind = True
nuclei = 'Argon'
for detector in detectors:
    for year in years:
        pcle = 'atmNu'+'_'+detector+'_'+year 
        
        recoil_type = get_recoil_type(pcle)
        bind = get_Ebind(pcle,recoil_type, ebind)
        
        pdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'pdf'),
                                    pcle+'_'+recoil_type+bind+nuclei+'_pdf.txt')
        cdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'cdf'),
                                pcle+'_'+recoil_type+bind+nuclei+'_cdf.txt')

        
        Er_GeV =get_Er(pcle, nuclei, recoil_type, start=start, binnum= binnum)
        Enu_MeV, dN_dEnu_percm2sMeV = flux_norm(pcle, 
                                                norm_unit = (u.cm**2 * u.s)**(-1), metallicity_model = '')
        
        pdfs = get_dR_dEr_pertonyrkeV(Er_GeV, nuclei, Enu_MeV.to(u.GeV), dN_dEnu_percm2sMeV, recoil_type, pcle,
                                                ebinding = ebind)
        
        good = [pdf[i]>pdf[i+1] for i in range(0, len(pdfs)-1)]
        pdfs = pdfs[:-1][good]
        
        dEr_GeVs = np.diff(Er_GeV)[good]
        Er_GeV_centers =((Er_GeV[1:]+Er_GeV[:-1])/2)[good]
        cdf = get_cdf(pdfs, dEr_GeVs, 'survival')
        Er_GeV = Er_GeV[:-1][good]
        E_centers = Er_GeV_centers.to(u.keV).value
        data_out_pdf = np.array([Er_GeV.to(u.keV).value, pdfs.value]).T
        data_out_cdf = np.array([E_centers, cdf/max(cdf)]).T
        print(pdf_path)
        print(cdf_path)
        write_file_data(pdf_path, data_out_pdf,10, append= False)
        write_file_data(cdf_path, data_out_cdf,10, append= False) 
        #pcle_tot_year_pdf.append(pdf)
        
    ''' 
    #pcle_avg_years_pdf= (pcle_tot_year_pdf[0]+pcle_tot_year_pdf[1])/2
    
    #data_out_pdf = np.array([Er_GeV.to(u.keV).value, pcle_avg_years_pdf.value]).T
    
    
    

    data_out_cdf = np.array([E_centers, cdf/max(cdf)]).T
    
    print(pdf_path, cdf_path)
    #write_file_data(pdf_path, data_out_pdf,10, append= False)
    #write_file_data(cdf_path, data_out_cdf,10, append= False)
    pdfs.append(pdf)
    Er_GeV_tot.append(Er_GeV)
    '''   

atmNu_CJPL_2009
atmNu_CJPL_2009
nucleon mass:  37.211326403360076 GeV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
survival
./real_data_nest/pdf/atmNu_CJPL_2009_NR_Argon_pdf.txt
./real_data_nest/cdf/atmNu_CJPL_2009_NR_Argon_cdf.txt
atmNu_CJPL_2014
atmNu_CJPL_2014
nucleon mass:  37.211326403360076 GeV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
survival
./real_data_nest/pdf/atmNu_CJPL_2014_NR_Argon_pdf.txt
./real_data_nest/cdf/atmNu_CJPL_2014_NR_Argon_cdf.txt
atmNu_CJPL_avg
atmNu_CJPL_avg
nucleon mass:  37.211326403360076 GeV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
survival
./real_data_nest/pdf/atmNu_CJPL_avg_NR_Argon_pdf.txt
./real_data_nest/cdf/atmNu_CJPL_avg_NR_Argon_cdf.txt
atmNu_Kamioka_2009
atmNu_Kamioka_2009
nucleon mass:  37.211326403360076 GeV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
survival
./real_data_nest/pdf/atmNu_Kamioka_2009_NR_Argon_pdf.txt
./rea

In [5]:

detectors = ['CJPL','Kamioka', 'LNGS', 'SURF','SNOlab']#
years = ['2009','2014']

pdfs = []
Er_GeV_tot = []
binnum = 400
start = -9
ebind = True
for detector in detectors:
    pcle_tot_year_pdf = []
    pcle_atm = 'atmNu'+'_'+detector+'_'+'avg'
    recoil_type = get_recoil_type(pcle_atm)
    bind = get_Ebind(pcle_atm,recoil_type, ebind)
    pdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'pdf'),
                                pcle_atm+'_'+recoil_type+bind+nuclei+'_pdf.txt')
    cdf_path = os.path.join(os.path.join(os.path.join(path, 'real_data_nest'),'cdf'),
                            pcle_atm+'_'+recoil_type+bind+nuclei+'_cdf.txt')
    for year in years:
        pcle = 'atmNu'+'_'+detector+'_'+year 
        Er_GeV =get_Er(pcle, nuclei, recoil_type,start, binnum)
      
        Enu_MeV, dN_dEnu_percm2sMeV = flux_norm(pcle)
        
   
    
        pdf = get_dR_dEr_pertonyrkeV(Er_GeV, nuclei, Enu_MeV.to(u.GeV), dN_dEnu_percm2sMeV, recoil_type, pcle,
                                                ebinding = ebind)
        pcle_tot_year_pdf.append(pdf)
    pcle_avg_years_pdf= (pcle_tot_year_pdf[0]+pcle_tot_year_pdf[1])/2
    
    data_out_pdf = np.array([Er_GeV.to(u.keV).value, pcle_avg_years_pdf.value]).T
    
    
    dEr_GeVs = np.diff(Er_GeV) 
    Er_GeV_centers =(Er_GeV[1:]+Er_GeV[:-1])/2
    cdf = get_cdf(pcle_avg_years_pdf[:-1], dEr_GeVs, 'survival')
    E_centers = Er_GeV_centers.to(u.keV).value

    data_out_cdf = np.array([E_centers, cdf/max(cdf)]).T
    print(pdf_path)
    write_file_data(pdf_path, data_out_pdf,10, append= False)
    write_file_data(cdf_path, data_out_cdf,10, append= False)
    pdfs.append(pdf)
    Er_GeV_tot.append(Er_GeV)
    

atmNu_CJPL_2009
atmNu_CJPL_2009
nucleon mass:  37.211326403360076 GeV
ebind:  [3206.2  3206.2   324.2   324.2   247.74  247.74  247.74  247.74  247.74
  247.74   29.24   29.24   15.76   15.76   15.76   15.76   15.76   15.76] eV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
atmNu_CJPL_2014
atmNu_CJPL_2014
nucleon mass:  37.211326403360076 GeV
ebind:  [3206.2  3206.2   324.2   324.2   247.74  247.74  247.74  247.74  247.74
  247.74   29.24   29.24   15.76   15.76   15.76   15.76   15.76   15.76] eV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
survival
atmNu_Kamioka_2009
atmNu_Kamioka_2009
nucleon mass:  37.211326403360076 GeV
ebind:  [3206.2  3206.2   324.2   324.2   247.74  247.74  247.74  247.74  247.74
  247.74   29.24   29.24   15.76   15.76   15.76   15.76   15.76   15.76] eV
6.02214076e+23 39.948 1.5074949334134373e+22 1 / g 37.211326403360076 GeV
atmNu_Kamioka_2014
atmNu_Kamioka_2014
nucleon mass:  37.211326403360076 GeV
ebi